In [1]:
import pandas as pd
from ProjectTools import OpenBeds
import math

dry = True

def filterDepth(df):
    average = df["readCount"].mean()
    df = df[df["readCount"].ge(10)]
    df = df[df["readCount"].le(average + 3*math.sqrt(average))]
    # df = df.loc[df["readCount"] == 15] # Testing constant readcount

    return df

if dry == True:
    wgbs_bed_path = './test_data/ENCSR893RHD_modifications_mm39_sub.bed'
    tab_bed_path = './test_data/CRR008807_TAB_merged.bedGraph.gz.bismark.zero.cov_sub.bed'
    oxbs_bed_path = './test_data/CRD018546.gz_val_1_bismark_bt2_pe.deduplicated.bedGraph.gz.bismark.zero.cov_sub.bed'

    nano_2mod_path = './test_data/prom_R10.4.1_E8.2_WGS_brain_DH_5mC_nonAggregated_sub.bedMethyl'
    nano_3mod_path = './test_data/prom_R10.4.1_E8.2_WGS_brain_0.9.1_mods_sub.bed'
    
    nano_mc_df, nano_hmc_df = map(filterDepth, OpenBeds.get_nanopore_threeMod(nano_3mod_path))

    nano_2mod_df = filterDepth(OpenBeds.get_nanopore_twoMod(nano_2mod_path))
    wgbs_df = filterDepth(OpenBeds.get_wgbs(wgbs_bed_path))
    tab_df = filterDepth(OpenBeds.get_bismark(tab_bed_path, "5hmC"))
    oxbs_df = filterDepth(OpenBeds.get_bismark(oxbs_bed_path, "5mC"))

else:
    wgbs_bed_path = './data/ENCSR893RHD_modifications_mm39.bed'
    tab_bed_path = './data/TAB_data/CRR008807_TAB_merged.bedGraph.gz.bismark.zero.cov'
    oxbs_bed_path = './data/CRR008808_oxBS_remergedRaw.zero.cov_modified.bed'

    nano_2mod_path = './data/prom_R10.4.1_E8.2_WGS_brain_DH_5mC_nonAggregated.bedMethyl'
    nano_3mod_path = './data/prom_R10.4.1_E8.2_WGS_brain_0.9.1_mods.bed'

    nano_mc_df, nano_hmc_df = map(filterDepth, OpenBeds.get_nanopore_threeMod(nano_3mod_path))

    nano_2mod_df = filterDepth(OpenBeds.get_nanopore_twoMod(nano_2mod_path))
    wgbs_df = filterDepth(OpenBeds.get_wgbs(wgbs_bed_path))
    tab_df = filterDepth(OpenBeds.get_bismark(tab_bed_path, "5hmC"))
    oxbs_df = filterDepth(OpenBeds.get_bismark(oxbs_bed_path, "5mC"))

In [40]:
def mergeMethods(bis_df, nano_df):
    wide_merged = pd.merge(
        bis_df, nano_df, "inner", ["chromosome", "chromStart", "chromEnd", "modification_type"], suffixes=["_Bisulphite", "_Nanopore"]
        )
    long_merged = pd.wide_to_long(wide_merged, stubnames=["readCount", "percentMeth", "method"], i=["chromosome", "chromStart", "chromEnd", "modification_type"], 
                                  j="Method", sep="_", suffix="\D+")
    return wide_merged.reset_index(), long_merged.reset_index()

In [48]:
from scipy import stats

stats.ks_2samp(hmc_df_wide["percentMeth_Bisulphite"], hmc_df_wide["percentMeth_Nanopore"], method="exact")

KstestResult(statistic=0.04969150503450491, pvalue=5e-323, statistic_location=0.0, statistic_sign=1)

In [46]:
import seaborn as sns
import matplotlib.pyplot as pl

sns.set_theme(style="whitegrid")
sns.set_palette("colorblind")

fig2 = pl.figure(figsize=(10, 15), dpi=600) 
gs = fig2.add_gridspec(nrows=3, ncols=2)

ax1 = fig2.add_subplot(gs[0, 0])
ax2 = fig2.add_subplot(gs[0, 1])
ax3 = fig2.add_subplot(gs[1, 0])
ax4 = fig2.add_subplot(gs[1, 1])
ax5 = fig2.add_subplot(gs[2, 0])
ax6 = fig2.add_subplot(gs[2, 1])

two_mod_wide, two_mod_long = mergeMethods(wgbs_df, nano_2mod_df)
sns.kdeplot(two_mod_long, x="percentMeth", hue="method", ax=ax1, fill=False)
sns.histplot(two_mod_long, x="percentMeth", hue="method", stat="density", binrange=(0, 100), bins=20, multiple="layer", ax=ax1)
sns.move_legend(ax1, "upper left", title=None)
ax1.set_xlabel("Percent modification (%) per CpG")
ax1.set_xlim([0, 100])
ax1.set_title("A", loc="left")

sns.ecdfplot(two_mod_long, x="percentMeth", hue="method", ax=ax2)
sns.move_legend(ax2, "upper left", title=None)
ax2.set_ylabel("Proportion of all CpGs")
ax2.set_xlabel("Percent modification (%) per CpG")
# ax1.set_xticks(np.arange(0, 120, 20), labels=["0.0", "0.2", "0.4", "0.6", "0.8", "1.0"]) # to make proportions instead of percentages 
ax2.set_title("B", loc="left")
ax2.set_xlim([0, 100])

mc_df_wide, mc_df_long  = mergeMethods(oxbs_df, nano_mc_df)
sns.kdeplot(mc_df_long, x="percentMeth", hue="method", common_norm=True, ax=ax3)
sns.histplot(mc_df_long, x="percentMeth", hue="method", stat="density", common_norm=True, binrange=(0, 100), bins=20, multiple="layer", ax=ax3)
sns.move_legend(ax3, "upper left", title=None)
ax3.set_title("C", loc="left")
ax3.set_xlabel("Percent modification (%) per CpG")
ax3.set_xlim([0, 100])

sns.ecdfplot(mc_df_long, x="percentMeth", hue="method", ax=ax4)
sns.move_legend(ax4, "upper left", title=None)
ax4.set_ylabel("Proportion of all CpGs")
ax4.set_xlabel("Percent modification (%) per CpG")
# ax1.set_xticks(np.arange(0, 120, 20), labels=["0.0", "0.2", "0.4", "0.6", "0.8", "1.0"]) # to make proportions instead of percentages 
ax4.set_title("D", loc="left")
ax4.set_xlim([0, 100])

hmc_df_wide, hmc_df_long = mergeMethods(tab_df, nano_hmc_df)
sns.kdeplot(hmc_df_long, x="percentMeth", hue="method", common_norm=True, ax=ax5)
sns.histplot(hmc_df_long, x="percentMeth", hue="method", stat="density", common_norm=True, binrange=(0, 100), bins=20, multiple="layer", ax=ax5)
sns.move_legend(ax5, "center", title=None)
ax5.set_title("E", loc="left")
ax5.set_xlabel("Percent modification (%) per CpG")
ax5.set_xlim([0, 100])

sns.ecdfplot(hmc_df_long, x="percentMeth", hue="method", ax=ax6)
sns.move_legend(ax6, "center", title=None)
ax6.set_ylabel("Proportion of all CpGs")
ax6.set_xlabel("Percent modification (%) per CpG")
ax6.set_xlim([0, 100])
ax6.set_title("F", loc="left")

fig2.tight_layout()